# Importing Libraries

In [50]:
import time
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import KFold, train_test_split, RepeatedKFold, cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
import statsmodels.formula.api as smf
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor, RandomForestClassifier, ExtraTreesClassifier
from sklearn import svm
#from xgboost import XGBRegressor
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
import category_encoders as ce

# reading csv files

In [27]:
col = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class_value']
df =  pd.read_table('car.data', delimiter= ',', header=None, names=col)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class_value
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


### Attribute Values:

#### buying:       v-high, high, med, low
#### maint:        v-high, high, med, low
#### doors:        2, 3, 4, 5-more
#### persons:      2, 4, more
#### lug_boot:     small, med, big
#### safety:       low, med, high

# Statistic Summary of df

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   price     1728 non-null   object
 1   buying    1728 non-null   object
 2   maint     1728 non-null   object
 3   doors     1728 non-null   object
 4   persons   1728 non-null   object
 5   lug_boot  1728 non-null   object
 6   safety    1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [32]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,class_value
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,high,high,2,2,small,high,unacc
freq,432,432,432,576,576,576,1210


## Frequency distribution of values in variables

In [33]:
for col in col:
    print(df[col].value_counts())   

high     432
vhigh    432
low      432
med      432
Name: buying, dtype: int64
high     432
vhigh    432
low      432
med      432
Name: maint, dtype: int64
2        432
5more    432
3        432
4        432
Name: doors, dtype: int64
2       576
more    576
4       576
Name: persons, dtype: int64
small    576
big      576
med      576
Name: lug_boot, dtype: int64
high    576
low     576
med     576
Name: safety, dtype: int64
unacc    1210
acc       384
good       69
vgood      65
Name: class_value, dtype: int64


# Split feature and target variables

In [34]:
X = df.drop(['class_value'], axis=1)
y = df['class_value']

# Split data into training and test set

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [36]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape

((1157, 6), (571, 6))

# Feature Engineering

## Categorical Variables Encoding

In [44]:
# encode categorical variables with LabelEncoder

encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])

X_train = encoder.fit_transform(X_train)

X_test = encoder.transform(X_test)

C:\Users\BOK\anaconda3\envs\test_env\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [45]:
X_train.head()

,buying,maint,doors,persons,lug_boot,safety
48,1,1,1,1,1,1
468,2,1,1,2,2,1
155,1,2,1,1,2,2
1721,3,3,2,1,2,2
1208,4,3,3,1,2,2


In [46]:
X_test.head()

,buying,maint,doors,persons,lug_boot,safety
599,2,2,4,3,1,2
1201,4,3,3,2,1,3
628,2,2,2,3,3,3
1498,3,2,2,2,1,3
1263,4,3,4,1,1,1


# Model Building

In [51]:
rfc = RandomForestClassifier(random_state=0)

# fit the model

rfc.fit(X_train, y_train)

# Predict the Test set results

y_pred = rfc.predict(X_test)

# Check accuracy score 

print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.9457
